In [1]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

Récupération des données

In [2]:
# Connexion à la base de données SQLite
db_path = '../../db.sqlite3'
conn = sqlite3.connect(db_path)

# Chargement des données
menu = pd.read_sql_query("SELECT * FROM menu", conn)
orders = pd.read_sql_query("SELECT * FROM orders", conn)

# Fermer la connexion
conn.close()

Préparation des données

In [3]:
order_data = orders.merge(menu, on='pizza_id')

# Remplacer pizza_id par name
order_data_grouped = order_data.groupby(['client_id', 'name'])['name'].count().unstack(fill_value=0)

# Création des labels pour recommandation
# La pizza la plus commandée pour chaque client devient la cible
order_data_grouped['most_ordered'] = order_data_grouped.idxmax(axis=1)
target = order_data_grouped['most_ordered']
features = order_data_grouped.drop(columns=['most_ordered'])

# Encodage des pizzas
features_encoded = features  # Pas besoin d'encodage pour les noms dans ce cas
target_encoded = target.astype('category').cat.codes
pizza_name_mapping = dict(enumerate(target.astype('category').cat.categories))  # Pour décoder plus tard

Entraînement

In [4]:
# Séparation des données pour entraînement et test
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target_encoded, test_size=0.2, random_state=42)

# Entraînement du modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)

# Analyse des performances
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=pizza_name_mapping.values()))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9014285714285715

Classification Report:
                                             precision    recall  f1-score   support

                The Barbecue Chicken Pizza       0.88      0.99      0.93       107
                        The Big Meat Pizza       0.96      0.97      0.97        78
                      The Brie Carre Pizza       1.00      0.48      0.65        21
                       The Calabrese Pizza       0.94      0.92      0.93        37
              The California Chicken Pizza       0.91      1.00      0.95        89
                 The Chicken Alfredo Pizza       0.91      0.81      0.86        37
                   The Chicken Pesto Pizza       1.00      0.80      0.89        25
                  The Classic Deluxe Pizza       0.80      0.93      0.86        74
                     The Five Cheese Pizza       0.91      0.98      0.95        44
                     The Four Cheese Pizza       0.79      0.94      0.86        32
                     

Exemple de recommandation

In [5]:
# Recommandation pour un nouveau client
def recommend_pizza(client_order_vector):
    client_vector_encoded = pd.DataFrame([client_order_vector], columns=features_encoded.columns).fillna(0)
    predicted_pizza_code = model.predict(client_vector_encoded)[0]
    predicted_pizza_name = pizza_name_mapping[predicted_pizza_code]
    return predicted_pizza_name

# Exemple d'utilisation pour un client avec un historique fictif
new_client_orders = [0] * len(features.columns)  # Exemple : toutes les commandes à 0
print("Pizza recommandée :", recommend_pizza(new_client_orders))


Pizza recommandée : The Pepperoni Pizza
